# 🐛 Pest Classification Model — Full Pipeline

**GPU-accelerated training, evaluation, quantization & benchmarking**

1. Upload `curated_data.zip` (from local machine)
2. Train EfficientNetB0 with transfer learning (20 + 10 epochs)
3. Evaluate top-1 & top-3 accuracy
4. Quantize to INT8 TFLite
5. Benchmark inference latency
6. Download `pest_model.tflite` + `labels.json`

> **Runtime → Change runtime type → T4 GPU** before running!

## 1️⃣ Upload & Extract Dataset

In [ ]:
from google.colab import files
import zipfile, os

print("Upload curated_data.zip from your local machine...")
uploaded = files.upload()

with zipfile.ZipFile('curated_data.zip', 'r') as z:
    z.extractall('.')

# Verify
for split in ['train', 'val', 'test']:
    n = sum(len(f) for _, _, f in os.walk(f'curated_data/{split}'))
    print(f'{split}: {n} images')

print('\n✅ Dataset ready!')

## 2️⃣ Setup & GPU Check

In [ ]:
import json
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix

print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

# Constants
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 18

# Load labels
with open('curated_data/labels.json') as f:
    LABELS = json.load(f)
CLASS_NAMES = [LABELS[str(i)] for i in range(NUM_CLASSES)]
print(f"\n{NUM_CLASSES} classes: {CLASS_NAMES}")

## 3️⃣ Load Datasets

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    'curated_data/train', image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE, label_mode='int', shuffle=True, seed=42)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'curated_data/val', image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE, label_mode='int', shuffle=False)

test_ds = tf.keras.utils.image_dataset_from_directory(
    'curated_data/test', image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE, label_mode='int', shuffle=False)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)

# Class weights
all_labels = []
for _, labels in train_ds:
    all_labels.extend(labels.numpy().tolist())
counts = np.bincount(all_labels, minlength=NUM_CLASSES)
total = len(all_labels)
class_weights = {i: total / (NUM_CLASSES * counts[i]) if counts[i] > 0 else 1.0 for i in range(NUM_CLASSES)}
print(f"\nTotal training images: {total}")
print(f"Class weight range: {min(class_weights.values()):.2f} – {max(class_weights.values()):.2f}")

## 4️⃣ Build Model

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False, weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

model.summary()

## 5️⃣ Phase 1: Train Classifier Head (base frozen)

In [ ]:
HEAD_EPOCHS = 20

callbacks_head = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ModelCheckpoint('pest_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1),
]

print(f"Phase 1: Training classifier head for {HEAD_EPOCHS} epochs...")
hist1 = model.fit(train_ds, validation_data=val_ds, epochs=HEAD_EPOCHS,
                  class_weight=class_weights, callbacks=callbacks_head)
print(f"\n✅ Phase 1 done — Best val_accuracy: {max(hist1.history['val_accuracy']):.4f}")

## 6️⃣ Phase 2: Fine-tune Last 20 Layers

In [ ]:
FT_EPOCHS = 10

base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

callbacks_ft = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ModelCheckpoint('pest_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1),
]

print(f"Phase 2: Fine-tuning last 20 layers for {FT_EPOCHS} epochs...")
hist2 = model.fit(train_ds, validation_data=val_ds, epochs=FT_EPOCHS,
                  class_weight=class_weights, callbacks=callbacks_ft)
print(f"\n✅ Phase 2 done — Best val_accuracy: {max(hist2.history['val_accuracy']):.4f}")

## 7️⃣ Training Curves

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
offset = 0
for name, hist, color in [('Head', hist1, '#2196F3'), ('Fine-tune', hist2, '#FF5722')]:
    epochs = range(offset+1, offset+len(hist.history['accuracy'])+1)
    ax1.plot(epochs, hist.history['accuracy'], '-o', color=color, label=f'{name} train', ms=3)
    ax1.plot(epochs, hist.history['val_accuracy'], '--s', color=color, label=f'{name} val', ms=3, alpha=0.7)
    ax2.plot(epochs, hist.history['loss'], '-o', color=color, label=f'{name} train', ms=3)
    ax2.plot(epochs, hist.history['val_loss'], '--s', color=color, label=f'{name} val', ms=3, alpha=0.7)
    offset += len(hist.history['accuracy'])

ax1.set_title('Accuracy'); ax1.set_xlabel('Epoch'); ax1.legend(); ax1.grid(alpha=0.3)
ax2.set_title('Loss'); ax2.set_xlabel('Epoch'); ax2.legend(); ax2.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('training_curves.png', dpi=150)
plt.show()

## 8️⃣ Evaluate on Test Set

In [ ]:
# Load best model
model = tf.keras.models.load_model('pest_model.keras')

all_labels, all_probs = [], []
for images, labels in test_ds:
    probs = model.predict(images, verbose=0)
    all_probs.append(probs)
    all_labels.extend(labels.numpy().tolist())

all_probs = np.concatenate(all_probs, axis=0)
all_labels = np.array(all_labels)
all_preds = np.argmax(all_probs, axis=1)

top1 = np.mean(all_preds == all_labels)
top3_preds = np.argsort(all_probs, axis=1)[:, -3:]
top3 = np.mean([all_labels[i] in top3_preds[i] for i in range(len(all_labels))])

print(f"Top-1 Accuracy: {top1:.4f} ({top1*100:.1f}%)")
print(f"Top-3 Accuracy: {top3:.4f} ({top3*100:.1f}%)")
print(f"Top-3 Target (>85%): {'✅ PASSED' if top3 > 0.85 else '❌ BELOW TARGET'}")
print()
print(classification_report(all_labels, all_preds, target_names=CLASS_NAMES, digits=3))

### Confusion Matrix

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
fig, ax = plt.subplots(figsize=(14, 12))
im = ax.imshow(cm, cmap='Blues')
fig.colorbar(im, ax=ax, shrink=0.8)
ax.set_xticks(range(NUM_CLASSES)); ax.set_xticklabels(CLASS_NAMES, rotation=45, ha='right', fontsize=8)
ax.set_yticks(range(NUM_CLASSES)); ax.set_yticklabels(CLASS_NAMES, fontsize=8)
thresh = cm.max() / 2
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, str(cm[i,j]), ha='center', va='center', fontsize=7,
                color='white' if cm[i,j] > thresh else 'black')
ax.set_ylabel('True'); ax.set_xlabel('Predicted'); ax.set_title('Confusion Matrix')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150)
plt.show()

## 9️⃣ Quantize to INT8 TFLite

In [ ]:
# Representative dataset for INT8 calibration
def representative_dataset():
    cal_ds = tf.keras.utils.image_dataset_from_directory(
        'curated_data/train', image_size=(IMG_SIZE, IMG_SIZE),
        batch_size=1, label_mode=None, shuffle=True, seed=42)
    for i, images in enumerate(cal_ds):
        yield [images.numpy().astype(np.float32)]
        if i >= 200: break

# Float32 conversion
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float = converter.convert()
with open('pest_model_float32.tflite', 'wb') as f:
    f.write(tflite_float)
print(f"Float32 TFLite: {len(tflite_float)/1024/1024:.1f} MB")

# INT8 conversion
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS_INT8,
    tf.lite.OpsSet.TFLITE_BUILTINS,
]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.float32

print("Quantizing with 200 calibration images...")
tflite_int8 = converter.convert()
with open('pest_model.tflite', 'wb') as f:
    f.write(tflite_int8)
print(f"INT8 TFLite: {len(tflite_int8)/1024/1024:.1f} MB")
print(f"Compression: {len(tflite_float)/len(tflite_int8):.1f}x")

### Validate Quantized Accuracy

In [ ]:
interpreter = tf.lite.Interpreter(model_path='pest_model.tflite')
interpreter.allocate_tensors()
inp = interpreter.get_input_details()
out = interpreter.get_output_details()

correct, total_count = 0, 0
for images, labels in test_ds:
    for i in range(images.shape[0]):
        img = images[i:i+1].numpy()
        if inp[0]['dtype'] == np.uint8:
            s, z = inp[0]['quantization']
            img = (img / s + z).astype(np.uint8)
        else:
            img = img.astype(np.float32)
        interpreter.set_tensor(inp[0]['index'], img)
        interpreter.invoke()
        pred = np.argmax(interpreter.get_tensor(out[0]['index'])[0])
        if pred == labels[i].numpy():
            correct += 1
        total_count += 1

int8_acc = correct / total_count
drop = (top1 - int8_acc) * 100
print(f"Keras accuracy:  {top1:.4f}")
print(f"INT8 accuracy:   {int8_acc:.4f}")
print(f"Drop:            {drop:.1f}%  {'✅ Within budget' if drop <= 3 else '⚠️ Exceeds 3%'}")

## 🔟 Benchmark Inference Latency

In [ ]:
# Benchmark on CPU (simulates mobile device)
interpreter = tf.lite.Interpreter(model_path='pest_model.tflite', num_threads=1)
interpreter.allocate_tensors()
inp = interpreter.get_input_details()
out = interpreter.get_output_details()

# Warmup
dummy = np.zeros((1, IMG_SIZE, IMG_SIZE, 3), dtype=inp[0]['dtype'])
for _ in range(5):
    interpreter.set_tensor(inp[0]['index'], dummy)
    interpreter.invoke()

# Timed runs
latencies = []
for images, _ in test_ds.take(4):  # ~128 images
    for i in range(images.shape[0]):
        img = images[i:i+1].numpy()
        if inp[0]['dtype'] == np.uint8:
            s, z = inp[0]['quantization']
            img = (img / s + z).astype(np.uint8)
        else:
            img = img.astype(np.float32)
        t0 = time.perf_counter()
        interpreter.set_tensor(inp[0]['index'], img)
        interpreter.invoke()
        _ = interpreter.get_tensor(out[0]['index'])
        latencies.append((time.perf_counter() - t0) * 1000)

latencies = np.array(latencies)
print(f"Inference Latency (single-thread CPU):")
print(f"  Mean:   {latencies.mean():.1f} ms")
print(f"  Median: {np.median(latencies):.1f} ms")
print(f"  P95:    {np.percentile(latencies, 95):.1f} ms")
print(f"  Target (<500ms): {'✅ MET' if latencies.mean() < 500 else '⚠️ EXCEEDED'}")

# Summary
f_size = os.path.getsize('pest_model_float32.tflite') / 1024 / 1024
q_size = os.path.getsize('pest_model.tflite') / 1024 / 1024
print(f"\n{'Model':<20} {'Size (MB)':>10} {'Latency (ms)':>13} {'Accuracy':>10}")
print('-'*56)
print(f"{'Keras (full)':20} {'—':>10} {'—':>13} {top1:>10.1%}")
print(f"{'TFLite Float32':20} {f_size:>10.1f} {'—':>13} {'—':>10}")
print(f"{'TFLite INT8':20} {q_size:>10.1f} {latencies.mean():>13.1f} {int8_acc:>10.1%}")

## 📦 Download Artifacts

In [ ]:
# Save benchmark report
report = {
    'keras_top1_accuracy': float(top1),
    'keras_top3_accuracy': float(top3),
    'int8_accuracy': float(int8_acc),
    'accuracy_drop_pct': float(drop),
    'int8_model_size_mb': float(q_size),
    'mean_latency_ms': float(latencies.mean()),
    'median_latency_ms': float(np.median(latencies)),
    'p95_latency_ms': float(np.percentile(latencies, 95)),
}
with open('benchmark_report.json', 'w') as f:
    json.dump(report, f, indent=2)

# Save labels.json at top level
import shutil
shutil.copy('curated_data/labels.json', 'labels.json')

print("📥 Downloading model and labels...")
files.download('pest_model.tflite')
files.download('labels.json')
files.download('benchmark_report.json')
files.download('training_curves.png')
files.download('confusion_matrix.png')
print("\n✅ All done! Files downloaded.")